#ARS TLS practical part 1: Automated Segmentation and Quantitative Structure Model (QSM) Generation

In the first part of the practical we want to turn our plot point cloud into tree point clouds (instance segmentation). We can do this automatically with a software toolbox called RayCloudTools.

##Step 0: Installing RayCloudTools and TreeTools
Simply run the following code blocks to begin the installation of RayCloudTools, TreeTools, and their dependencies. This will take ~5-10 minutes.

*Note that this differs slightly from the normal installation process to ensure it works in the google colab environment

In [1]:
# ==========================================================
# Mount Google Drive (run once per session)
# ==========================================================

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/
!sudo apt-get install libeigen3-dev
!git clone https://github.com/ethz-asl/libnabo.git
%cd libnabo
!ls
!git checkout tags/1.0.7
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE=RelWithDebInfo
!make
!sudo make install
%cd ../..
!git clone http://github.com/qhull/qhull.git
%cd qhull
!git checkout tags/v7.3.2
!mkdir build
%cd build
!cmake .. -DCMAKE_POSITION_INDEPENDENT_CODE:BOOL=true
!make
!sudo make install
%cd ../..
!ls
!git clone https://github.com/csiro-robotics/raycloudtools.git
%cd raycloudtools
!mkdir build
%cd build
!cmake .. -DWITH_QHULL=ON
!sudo make install
%cd ../..
#extra step to modify cmake target file for google colab file structure, allows treetools to find raycloudtools install
with open("/content/raycloudtools/build/raycloudtools-config.cmake","r",encoding='utf-8') as file:
  data = file.readlines()
data[27] = 'include("/content/raycloudtools/build/raylib/raycloudtools-targets.cmake")\n'
with open("/content/raycloudtools/build/raycloudtools-config.cmake","w",encoding='utf-8') as file:
  file.writelines(data)
!git clone https://github.com/csiro-robotics/treetools.git
%cd treetools
!mkdir build
%cd build
!cmake .. -DCMAKE_PREFIX_PATH=/content/raycloudtools/build/
!sudo make install
%cd ../..

## Step 1: rayimport
**rayimport** is a function that will 'import' the data to a format that is usable with the raycloudtools (RCT) library. This will ensure that all the tools can be applied successfully to it.

Lets take a moment to examine the format of this function, as most of the tools within RCT will have a similar format. The following code block will call `rayimport` without its required parameters in order to display its description and options.

**If you are ever interested in the options available within the RCT library, simply calling the tool without any input will display the description of the tool and it's input options.**

In [ ]:
#change directory in drive
%cd /content/
#call rayimport with no inputs to get details
!/content/raycloudtools/build/bin/rayimport

Now that we have examined rayimport closer, we can use it to import our plot data. The following code block will do just that. Curious about the parameters used as input? You can always reference the previous code block to have a description. From now on, we will run the tools without displaying the description first, but you can always create your own code block and call a tool to see the details.

Using the demo data, this next block should run almost instantly. If you are seeing the description of rayimport again, it means there is an error (most likely the script cannot access the data successfully for some reason). Make sure the plot point cloud you want to process is within the data folder in your google drive, and its name and path exactly match the input of rayimport.

*Note:* If you see an error that says "Error: ray cloud has no identified points; all rays are zero-intensity non-returns,
many functions will not operate on this degerenate case." This is ok for now, just double check that it made a new file in the data folder on your google drive with '_raycloud' added to the name.

In [ ]:
#call rayimport
%cd /content/
!/content/raycloudtools/build/bin/rayimport /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm.ply ray 0,0,-0.1 --max_intensity 0

Now that we have imported the data, you can see a new file created in your google drive. Using the demo data as an example, this will be called 'Plot_ARS_5cm_raycloud.ply'

You can expect RCT to always follow a consistent naming convention. This raycloud file is what we will use for the next step.

## Step 2: rayextract terrain & trees

It's time to apply some tools that will extract the features we are interested in from our plot point cloud. This is accomplished in two different calls of the 'rayextract' tool; rayextract terrain and rayextract trees.

Lets first start with rayextract terrain. This step builds a mesh from the ground points in the point cloud that will be used for segmentation later:

In [ ]:
#call rayextract terrain
!/content/raycloudtools/build/bin/rayextract terrain /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm_raycloud.ply

> 🤔 **Reflection 1**: rayextract terrain is used for isolating the terrain in the plot, and has some limited optional parameters. How could different terrain forms change this process? Would you expect higher slopes or rougher terrain to cause difficulties during data processing? Is there a way we can minimize this while collecting our data?

You can see this saved a mesh file in your google drive data folder. If you're curious you can download this file from your drive, open CloudCompare and drag it into there to see what the terrain mesh looks like. 

Now lets call rayextract trees using both our raycloud and terrain mesh as inputs. Remember that you can call rayextract trees with no inputs to read about the other optional parameters we are using.

In [ ]:
#call rayextract trees
!/content/raycloudtools/build/bin/rayextract trees /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm_raycloud.ply /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm_raycloud_mesh.ply --max_diameter 0.9 --height_min 10

Our rayextract trees step has created a few new files for us, lets list them to keep track, as the naming can easily become confusing:
1. A trees_mesh.ply file that contains the mesh structures of the segmented trees. This file is very useful for visualization. If you are curious, you can download this file and open it in cloud compare.
2. A segmented.ply file that contains the point cloud of the plot with the individual trees segmented into separate classes. Again, feel free to open this file in cloud compare and check how the segmentation went.
3. A trees.txt file that contains a single line of structure data for each segmented tree in a format specific to RCT. Feel free to download and open this file if you are curious (be warned, it can be very large). While it looks overwhelming at first, this file contains information for each generated geometric primitive in the modeled trees.

To summarize, we now have a few different files that can be used in different ways. The mesh is great for visuals, the segmented point clouds can be further processed, and the trees.txt file already has useful data on the modeled structure of our trees.

> 🤔 **Reflection 2**: At this point we already have a huge amount of results with relatively low processing time - point clouds, meshes, and structural data. You can take all of these data types in multiple directions for analysis. What are some ideas you have for how we can further use these products? Can you use multiple of them at once for even stronger analysis, or pair them with other data types?

## Step 3: Splitting our files with raysplit and treesplit

rayextract is great, but it still leaves us with plot level data, and we are interested in examining individual trees. We can use raysplit to extract individual trees from the segmented.ply file (part of RCT), and treesplit to extract individual trees from the trees.txt file (part of the treetools library we installed at the beginning of this workbook).

If you are running into errors with folders not existing, it can be helpful to wait a minute between these blocks to help google drive catch up to google colab.


In [27]:
#Create new folders within the google drive to help us keep organized for the next steps
!mkdir /content/drive/MyDrive/ARS_TLS_course/data/individual_pointclouds
!mkdir /content/drive/MyDrive/ARS_TLS_course/data/individual_trees
!mkdir /content/drive/MyDrive/ARS_TLS_course/data/individual_meshes

In [ ]:
#call raysplit
!/content/raycloudtools/build/bin/raysplit /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm_raycloud_segmented.ply seg_colour

#move generated point clouds to the individual pointclouds folder
%cd /content/drive/MyDrive/ARS_TLS_course/data/
!mv *segmented_* /content/drive/MyDrive/ARS_TLS_course/data/individual_pointclouds
%cd /content/

In [ ]:
#call treesplit
!/content/treetools/build/bin/treesplit /content/drive/MyDrive/ARS_TLS_course/data/Plot_ARS_5cm_raycloud_trees.txt per-tree

#move generated tree files to the individual trees folder
%cd /content/drive/MyDrive/ARS_TLS_course/data/
!mv *_trees_* /content/drive/MyDrive/ARS_TLS_course/data/individual_trees
%cd /content/drive/MyDrive/ARS_TLS_course/data/individual_trees/
!mv *mesh* ..
%cd /content/

Now we have our individual trees and point clouds separated out in their respective folders. Luckily, the index number for each tree is coordinated to match between raysplit and treesplit, meaning the tree numbers should be the same.

It is now time to download the individual_trees and individual_pointclouds folders and inspect/manipulate them further in **CloudCompare**.

When you download the data, you may notice that there is a segmented_-1.ply point cloud generated, but no -1 tree.txt file. This -1 point cloud is the 'remainder' point cloud - that is all the points that were not selected as part of a tree in the plot.

> 🤔 **Reflection 3**: As you move to examine your segmented trees further in the lesson, you will see some errors in the automated segmentation and structural models. What aspects of the TLS data and forest do you think could be contributing to these errors in modeling? How would you expect other forest types and weather conditions to alter the results of this process? How would you design an experiment to test these questions?